In [1]:
import pandas as pd

In [2]:
dataset_path = r"C:\Users\User\PycharmProjects\lufthansa_task\lufthansa_task\cleaned_dataset"

In [3]:
# Import datasets
customers = pd.read_csv(dataset_path + '/customers.csv')
order_items = pd.read_csv(dataset_path +'/order_items.csv')
order_payments = pd.read_csv(dataset_path +'/order_payments.csv')
order_reviews = pd.read_csv(dataset_path +'/order_reviews.csv')
orders = pd.read_csv(dataset_path +'/orders.csv')
products = pd.read_csv(dataset_path +'/products.csv')
sellers = pd.read_csv(dataset_path +'/sellers.csv')

data = {'customers': customers,
        'order_items': order_items, 
        'order_payments': order_payments,
        'order_reviews': order_reviews,
        'orders': orders,
        'products': products,
        'sellers': sellers}

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\User\\PycharmProjects\\lufthansa_task\\lufthansa_task\\cleaned_dataset/customers.csv'

1) Total Price: Sum of product price and freight value.

In [40]:
# We will create a new column called total price. 
order_items['total_price'] = order_items['price'] + order_items['freight_value']

In [41]:
order_items[['total_price', 'price', 'freight_value']].head()

,total_price,price,freight_value
0,72.19,58.90,13.29
1,259.83,239.90,19.93
2,216.87,199.00,17.87
3,25.78,12.99,12.79
4,218.04,199.90,18.14


2) Delivery Time: Difference between delivery date and purchase date

In [42]:
# First we want to set the purchase and delivery to datetime.  
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])

In [43]:
# Next we want to merge the order_items table with order table to get the purchase and delivery date.
order_items = order_items.merge(orders[['order_id', 'order_purchase_timestamp', 'order_delivered_customer_date']], on='order_id', how='left')

In [44]:
# Calculate delivery time in days
order_items['delivery_time'] = (order_items['order_delivered_customer_date'] -order_items['order_purchase_timestamp']).dt.days


In [45]:
order_items[['order_id', 'delivery_time']].head()

,order_id,delivery_time
0,00010242fe8c5a6d1ba2dd792cb16214,7
1,00018f77f2f0320c557190d7a144bdd3,16
2,000229ec398224ef6ca0657da4fc703e,7
3,00024acbcdf0a6daa1e931b038114c75,6
4,00042b26cf59d7ce69dfabb4e55b4fd9,25


3) Payment Count: Sum of payment installments for each order.

In [46]:
# We will group by order_id and sum the payment installments
payment_sum = order_payments.groupby('order_id')['payment_installments'].sum().reset_index()
payment_sum.columns = ['order_id', 'total_payments_count']


In [47]:
# Merge the payment data into the order_items table
order_items = order_items.merge(payment_sum, on='order_id', how='left')
order_items[['order_id', 'total_payments_count']].head()


,order_id,total_payments_count
0,00010242fe8c5a6d1ba2dd792cb16214,2.0
1,00018f77f2f0320c557190d7a144bdd3,3.0
2,000229ec398224ef6ca0657da4fc703e,5.0
3,00024acbcdf0a6daa1e931b038114c75,2.0
4,00042b26cf59d7ce69dfabb4e55b4fd9,3.0


 4. Profit Margin: Subtract freight value from product price

In [48]:
# We will substract freight value from the product price
order_items['profit_margin'] = order_items['price'] - order_items['freight_value']

In [49]:
# Get all the new columns
order_items[['order_id', 'total_price', 'delivery_time', 'total_payments_count', 'profit_margin']].head()

,order_id,total_price,delivery_time,total_payments_count,profit_margin
0,00010242fe8c5a6d1ba2dd792cb16214,72.19,7,2.0,45.61
1,00018f77f2f0320c557190d7a144bdd3,259.83,16,3.0,219.97
2,000229ec398224ef6ca0657da4fc703e,216.87,7,5.0,181.13
3,00024acbcdf0a6daa1e931b038114c75,25.78,6,2.0,0.20
4,00042b26cf59d7ce69dfabb4e55b4fd9,218.04,25,3.0,181.76


In [50]:
order_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110189 entries, 0 to 110188
Data columns (total 13 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_id                       110189 non-null  object        
 1   order_item_id                  110189 non-null  int64         
 2   product_id                     110189 non-null  object        
 3   seller_id                      110189 non-null  object        
 4   shipping_limit_date            110189 non-null  object        
 5   price                          110189 non-null  float64       
 6   freight_value                  110189 non-null  float64       
 7   total_price                    110189 non-null  float64       
 8   order_purchase_timestamp       110189 non-null  datetime64[ns]
 9   order_delivered_customer_date  110189 non-null  datetime64[ns]
 10  delivery_time                  110189 non-null  int64         
 11  

In [52]:
order_items.to_csv(r"C:\Users\User\PycharmProjects\lufthansa_task\lufthansa_task\cleaned_dataset\order_items.csv", index=False)